In [1]:
import tensorflow as tf
from keras.applications.mobilenet import MobileNet
from keras.preprocessing import image

%pylab inline

Using TensorFlow backend.


Populating the interactive namespace from numpy and matplotlib


In [2]:
from keras.applications.mobilenet import decode_predictions, preprocess_input
from keras.layers import Input, Lambda, Flatten, Dense, Dropout, Cropping2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model
import keras.backend as K

TARGET_SHAPE = (160, 160)
TARGET_SIZE = (TARGET_SHAPE[0] + 160, TARGET_SHAPE[1])
BATCH_SIZE=128

def make_model():
    x = Input(shape=(TARGET_SIZE[0], TARGET_SIZE[1], 3))
    cropped = Cropping2D(((120, 40), (0, 0)))(x)
    preprocessed = Lambda(lambda x: preprocess_input(x))(cropped)
    base_model = MobileNet(alpha=1.0, include_top=False, input_shape=(TARGET_SHAPE[0], TARGET_SHAPE[1], 3))
    for layer in base_model.layers:
        layer.trainable = False

    model_out = base_model(preprocessed)
    flatten = Flatten()(model_out)
    dense1 = Dense(256, activation="elu")(flatten)
    norm1 = Dropout(0.2)(
        BatchNormalization()(dense1))
    dense2 = Dense(128, activation="elu")(norm1)
    norm2 = Dropout(0.2)(
        BatchNormalization()(dense2))
    dense3 = Dense(1)(norm2)
    
    model = Model(x, dense3)
    return model

model = make_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 320, 160, 3)       0         
_________________________________________________________________
cropping2d_1 (Cropping2D)    (None, 160, 160, 3)       0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 160, 160, 3)       0         
_________________________________________________________________
mobilenet_1.00_160 (Model)   (None, 5, 5, 1024)        3228864   
_________________________________________________________________
flatten_1 (Flatten)          (None, 25600)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               6553856   
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
__________

In [3]:
import os
import pandas as pd
import numpy as np
from keras.preprocessing import image
from PIL import Image

def _make_generator(df, batch_size=BATCH_SIZE):
    batch_X = []
    batch_y = []
    while True:
        for i, row in df.sample(frac=1.).iterrows():
            img = image.load_img(row[0], target_size=TARGET_SIZE)
            if row['flipped']:
                img = img.transpose(Image.FLIP_LEFT_RIGHT)
                batch_y.append(-row[3])
            else:
                batch_y.append(row[3])
                
            batch_X.append(image.img_to_array(img))
            
            if len(batch_X) >= batch_size:
                yield np.array(batch_X, dtype=np.float32), np.array(batch_y, dtype=np.float32)
                batch_X = []
                batch_y = []

        if batch_X:
            yield np.array(batch_X, dtype=np.float32), np.array(batch_y, dtype=np.float32)
            batch_X = []
            batch_y = []


def _load_path(path):
    data = pd.read_csv(path)
    data['flipped'] = np.zeros(len(data), dtype=np.bool)
    flipped_data = pd.read_csv(path)
    flipped_data['flipped'] = np.ones(len(flipped_data), dtype=np.bool)
    data = data.append(flipped_data)
    
    def _make_full_path(x):
        try:
            return os.path.join(os.path.dirname(path), x)
        except AttributeError:
            return x

    data = data.applymap(_make_full_path)
    return data

def data_generators(paths):
    data = pd.concat(_load_path(p) for p in paths)
    rnd = np.random.rand(len(data))
    train = data[rnd < 0.7]
    validation = data[(rnd >=0.7) & (rnd < 0.9)]
    test = data[rnd >= 0.9]
    
    return {
        "train": (len(train), _make_generator(train)),
        "valid": (len(validation), _make_generator(validation)),
        "test": (len(test), _make_generator(test))
    }

generators = data_generators([
    "/home/orazaev/workspace/data/data/driving_log.csv",
    "/home/orazaev/workspace/data/curves_data/driving_log.csv",
    "/home/orazaev/workspace/data/bridge/driving_log.csv"
])

In [4]:
model.compile(optimizer="adam", loss="mse")
train_size, train_gen = generators["train"]
valid_size, valid_gen = generators["valid"]
model.fit_generator(train_gen, train_size / BATCH_SIZE, epochs=15, validation_data=valid_gen, validation_steps=valid_size / BATCH_SIZE)

Epoch 1/15
159/158 [==============================] - 166s - loss: 0.6242 - val_loss: 0.0478
Epoch 2/15
159/158 [==============================] - 165s - loss: 0.2294 - val_loss: 0.0205
Epoch 3/15
159/158 [==============================] - 163s - loss: 0.1175 - val_loss: 0.0145
Epoch 4/15
159/158 [==============================] - 163s - loss: 0.0614 - val_loss: 0.0121
Epoch 5/15
159/158 [==============================] - 165s - loss: 0.0335 - val_loss: 0.0127
Epoch 6/15
159/158 [==============================] - 164s - loss: 0.0213 - val_loss: 0.0128
Epoch 7/15
159/158 [==============================] - 164s - loss: 0.0157 - val_loss: 0.0119
Epoch 8/15
159/158 [==============================] - 164s - loss: 0.0133 - val_loss: 0.0122
Epoch 9/15
159/158 [==============================] - 164s - loss: 0.0120 - val_loss: 0.0107
Epoch 10/15
159/158 [==============================] - 164s - loss: 0.0110 - val_loss: 0.0108
Epoch 11/15
159/158 [==============================] - 164s - loss: 0

In [5]:
test_size, test_gen = generators["test"]
model.evaluate_generator(test_gen, test_size / 32)

0.011815854086438952

In [6]:
model.save("drivenet.h5")